In [0]:
import random
def dh_public_parameters(b: Integer)->tuple[Integer, Integer]:
 
  public_parameter=[]
  while true:
   q=(random_prime(2^(b - 1),2^(b-2)))
   
   p=2*q+1
   if (is_prime(p) and ceil(math.log2(p))==b):
        break
  
  F= GF(p)
  g = F.multiplicative_generator()
  return p,g
number_of_bits = 128 # you can use a smaller bits, for testing e.g. 4 bit should yield 11 which is a safe prime, 2*5 + 1
safe_prime, primitive_element = dh_public_parameters(number_of_bits)
print(safe_prime, primitive_element)

In [0]:
import random
def dh_keypair(safe_prime: Integer, primitive_element: Integer) -> tuple[Integer, Integer]:

    
    private_key = random.randint(1, safe_prime - 1)
   
    public_key = pow(primitive_element, private_key , safe_prime)
    return [ private_key , public_key ]

alice_private_key, alice_public_key = dh_keypair(safe_prime, primitive_element)
bob_private_key, bob_public_key = dh_keypair(safe_prime, primitive_element)
print(alice_private_key, alice_public_key)
print(bob_private_key, bob_public_key)

In [0]:
def dh_sharedkey(safe_prime: Integer, primitive_element: Integer, own_private_key: Integer, other_public_key: Integer) -> Integer:
  
    private_shared_key= pow(other_public_key, own_private_key, safe_prime)
    return private_shared_key

alice_shared_key = dh_sharedkey(safe_prime, primitive_element, alice_private_key, bob_public_key)
bob_shared_key = dh_sharedkey(safe_prime, primitive_element, bob_private_key, alice_public_key)
print(alice_shared_key,bob_shared_key )
assert alice_shared_key == bob_shared_key

In [0]:
number_of_bits = 512 # this can encrypt messages m up to 2**512
p= random_prime(2^(number_of_bits//2),2^(number_of_bits//2 )+1)
q =random_prime(2^(number_of_bits),2^(number_of_bits//2+1))

n = p*q
while (p == q or ceil(math.log2(n)<number_of_bits) ):
        q = random_prime(2^(number_of_bits),2^(number_of_bits//2+1))
        n = p*q


e = 65537

public_key = n, e
print('public_key',public_key)

In [0]:
message = ZZ.random_element(1, 2**number_of_bits)
print('message:',message)
n,e=public_key
cipher = pow(message,e,n)
print('cipher',cipher)

In [0]:
phi_n = (p - 1) * (q - 1)
d = inverse_mod(e, phi_n)
private_key=n,d
decrypted_message = pow(cipher,d,n)
print(decrypted_message )
assert message == decrypted_message

In [0]:
d_p=d%(p-1)
d_q=d%(q-1)
q_p =pow(q,-1,p)

m1 =pow(cipher,d_p,p)
m2 = pow(cipher,d_q,q)
m=crt(m1,m2,p,q)
print('m:',m)

In [0]:
import hashlib

message = "test message"
print(hashlib.sha256().digest_size) # in bytes
hash = int(hashlib.sha256(message.encode()).hexdigest(), 16)
print(hash)

In [0]:
import hashlib
def find_generator(p, q):
    for g in range(2, p):
        if pow(g, q, p) == 1:  # g^q mod p = 1
            
            return g
    raise ValueError("No valid generator found!")

number_of_bits = 8 * hashlib.sha256().digest_size

# Step 1: Generate q  and p
q = random_prime(2^number_of_bits  - 1, 2^(number_of_bits  - 1))
print('q:',q)
k=2
while True:
     p = k * q + 1
     if is_prime(p):
            break
     k +=1
print('p:',p)


# Step 2: Select a primitive element 
g =find_generator(p,q)
print('g:',g)  

# Step 3: Generate private key x and public key y
x = randint(1, q - 1)  
private_key = x
print('x:',x)

# Step 4: Generate public key y
y = pow(g, x, p) 
public_key=p,q,g,y
print('y:',y)

In [0]:
import random
message = 'egy aprócska kalapocska, benne csacska macska mocska'
# Step 1: Choose a random k
k = random.randint(1, q - 1)

# Step 2: Compute r
r = mod(pow(g, k, p), q)

# Step 3: Compute the modular inverse k_inv
k_inv = inverse_mod(k, q)

#  Hash the message 
h =  int(hashlib.sha256(message.encode()).hexdigest(), 16) %q

# Step 4: Compute  s
s =( k_inv * (h + x * r)) % q
signature=r,s
print('signature:',signature)

In [0]:
from sympy import mod_inverse
r, s = signature
p, q, g,y = public_key
# Step 1: Compute w = s^(-1) mod q
w=mod_inverse(s, q)
    
# Step 2: Compute the hash of the message and u1
h = int(hashlib.sha256(message.encode()).hexdigest(), 16) % q

    
u1 = (h * w) % q

# Step 3: Compute  u2
u2 = (r * w) % q
    
# Step 4: Compute v = (g^u1 * y^u2 mod p) mod q
v = (pow(g, u1, p) * pow(y, u2, p)) % p % q
    
print(v==r)